# Data Challenge: [Help a Hematologist out!](https://helmholtz-data-challenges.de/web/challenges/challenge-page/93/overview) 
*** 
<b> Group: </b> 
> #      $ BLAMAD $  
<b> members </b> 
> Bashir K., 
> Lea G., 
> Ankita N., 
> Martin B., 
> Arnab M., 
> Dawit H. 
***

![logo](https://github.com/christinab12/Data-challenge-logo/blob/main/logo.jpg?raw=true)

## Getting started


This notebook is a short summary for getting started with the challenge ( found [here](https://helmholtz-data-challenges.de/web/challenges/challenge-page/93/overview)  ). Below you can find how to download the dataset and also the different labels along with exploring and analyzing the input and output data of the challenge, running a baseline model and creating a submission file to upload to the leaderboard.

***

<b>dataset:</b>

Three datasets, each constituting a different domain, will be used for this challenge:
> 1. The Acevedo_20 dataset with labels
> 2. The Matek_19 dataset with labels
> 3. The WBC dataset <b> without labels </b> (Used for domain adaptation and performance measurement)

The Acevedo_20 and Matek_19 datasets are labeled and should be used to train the model for the domain generalization task.
A small subpart of the WBC dataset, WBC1, will be downloadable from the beginning of the challenge. It is unlabeled and should be used for evaluation and domain adaptation techniques.

A second similar subpart of the WBC dataset, WBC2, will become available for download during phase 2 of the challenge, i.e. on the last day, 24 hours before submissions close.

***
<b>Goal: </b> 

The challenge here is in transfer learning, <b> precisely domain generalization (DG) and domain adaptation (DA) </b> techniques. The focus lies on using deep neural networks to classify single white blood cell images obtained from peripheral blood smears.
<b> Tthe goal of this challenge is to achieve a high performance, especially a high f1 macro score, on the WBC2 dataset. </b>

***
<b>Notes: </b>

This challenge wants to motivate research in domain generalization and adaptation techniques:

To make actual use of deep learning in the medical routine, it is important that the techniques can be used in realistic cases. If a peripheral blood smear is acquired from a patient and classified by a neural network, it is important that this works reliably. But the patient’s blood smear might very likely vary compared to the image domains used as training data of the network, resulting in not trustable results. To overcome this obstacle and build robust domain-invariant classifiers research in domain generalization and adaptation is needed.

***
<b>f1_score: </b>
[wikepedia](https://en.wikipedia.org/wiki/F-score)

> sklearn.metrics.f1_score(y_true, y_pred, *, labels=None, pos_label=1,<b> average='macro' </b>, sample_weight=None, zero_division='warn')

The formula can be see in [click here for the code](https://github.com/scikit-learn/scikit-learn/blob/36958fb24/sklearn/metrics/_classification.py#L1001) and is given as

> <g> F1 = 2 * (precision * recall) / (precision + recall) </g>

<img src="/beegfs/desy/user/hailudaw/blamad/figures/f1_score.PNG" width="384" height="681">

***


<b> Donwloading the data </b>

Uncomment the code below to download the dataset. Makesure you adjust the path according to where you download it

 

In [ ]:
# !wget --user YraZEdrHytaCSza --password BgZL3j8DT4 https://hmgubox2.helmholtz-muenchen.de/public.php/webdav/Acevedo_20.zip -O Acevedo_20.zip #(230M) [application/zip]
# !wget --user YraZEdrHytaCSza --password BgZL3j8DT4 https://hmgubox2.helmholtz-muenchen.de/public.php/webdav/Matek_19.zip -O Matek_19.zip #(5.7G) [application/zip]
# !wget --user YraZEdrHytaCSza --password BgZL3j8DT4 https://hmgubox2.helmholtz-muenchen.de/public.php/webdav/WBC1.zip -O WBC1.zip #(357M) [application/zip]
# !wget --user YraZEdrHytaCSza --password BgZL3j8DT4 https://hmgubox2.helmholtz-muenchen.de/public.php/webdav/val_dummy.csv -O val_dummy.csv #44834 (44K) [text/csv]
# !wget --user YraZEdrHytaCSza --password BgZL3j8DT4 https://hmgubox2.helmholtz-muenchen.de/public.php/webdav/metadata2.csv -O metadata2.csv #2019059 (1.9M) [text/csv]
# print('download complete') 

# import shutil

# shutil.unpack_archive('Acevedo_20.zip', 'Datasets/Acevedo_20')
# shutil.unpack_archive('Matek_19.zip', 'Datasets/Matek_19')
# shutil.unpack_archive('WBC1.zip', 'Datasets/WBC1')
# !ls

<b> datapath </b>

In [ ]:
import os
Dataset_path = os.getcwd() + '/Datasets'
data_path = { 'Ace_20': Dataset_path + '/Acevedo_20',
               'Ace_20_noisy': '/beegfs/desy/user/hailudaw/challenge/Datasets/Acevedo_20_noisy',
              'Mat_19': Dataset_path + '/Matek_19',
                'Mat_19_noisy': '/beegfs/desy/user/hailudaw/challenge/Datasets/Matek_19_noisy',
              'WBC1': Dataset_path + '/WBC1'
            }   

<b> labels </b>

In [ ]:
# Common classes of the datasets and their labels: 
# Highly underrepresented classes like atypical lymphocytes and smudge cells were left out.

label_map_all = {'basophi': 0, 'eosinophi': 1, 'erythroblast': 2, 'myeloblast' : 3, 'promyelocyte': 4, 'myelocyte': 5, 'metamyelocyte': 6, 'neutrophil_banded': 7, 'neutrophil_segmented': 8, 'monocyte': 9, 'lymphocyte_typical': 10}
label_map_reverse = { 0: 'basophil', 1: 'eosinophil', 2: 'erythroblast', 3: 'myeloblast', 4: 'promyelocyte', 5: 'myelocyte', 6: 'metamyelocyte', 7: 'neutrophil_banded', 8: 'neutrophil_segmented', 9: 'monocyte', 10: 'lymphocyte_typical'}

# The unlabeled WBC dataset gets the classname 'Data-Val' for every image
label_map_pred = {'DATA-VAL': 0 }

In [ ]:
import data_processing as dp 
'''includes the data processing functions like  finding_classes, metadata_generator, compute_mean, data_report, data_plot, crop (data_loader, data_augmentation, data_preprocessing, data_splitting, data_normalization, data_visualization,)'''
import dataset_generator as dg
'''includes the dataset generator functions like  dataset_generator, dataset_generator_pred'''

In [ ]:
print("Labels for the datapath Ace_20: ", dp.finding_classes(data_path['Ace_20']))

metadata = dp.metadata_generator(data_path)

In [ ]:
print((metadata.values[:,1]).tolist()[0:10])

In [ ]:
import skimage.io as io
import matplotlib.pyplot as plt

image = io.imread(metadata.values[:,1].tolist()[0])
image2 = io.imread(metadata.values[:,1].tolist()[15000])
print(image.shape)

for ch in range(3):
    plt.subplot(1,3,ch+1, title='Channel {}'.format(ch+1, image.shape), xticks=[], yticks=[])
    plt.imshow(image[:,:,ch])
    plt.plot(image[:,:,ch][:,image.shape[1]//2], color='gray', linewidth=0.5)
    plt.plot(image[:,:,ch][:,image.shape[0]//2], color='gray', linewidth=0.5)
    plt.axis('off')
plt.show()


<b> convert the dataset to a Pandas frame and compute the mean </b> 

In [ ]:
import tqdm
import numpy as np
import pandas as pd

#let's look at some of the statistics of the dataset. and later on, add it to the metadata
for ch in range(3):
    print('Channel {}:'.format(ch+1))
    print('Mean: ', np.mean(image[:,:,ch]))
    metadata.values[:,5+ch] = np.mean(image[:,:,ch])
    print('Std: ', np.std(image[:,:,ch]))
    print('Max: ', np.max(image[:,:,ch]))
    print('Min: ', np.min(image[:,:,ch]))
    print('')

added = pd.DataFrame(columns=["Image", "file", "label", "dataset", "set", 'mean1', 'mean2', 'mean3', 'std1', 'std2', 'std3', 'max1', 'max2', 'max3', 'min1', 'min2', 'min3'])
metadata = pd.concat([metadata, added], axis=0)
# dp.make_stat(metadata, savepath='metadata3.csv')

<b> Updating the panda frame with the different stat measure </b>

In [103]:
from distutils.log import error
import rasterio as rio
def make_stat(metadata = metadata, savepath = 'metadata3.csv'):
    for idx in tqdm.tqdm(range(int(len(metadata)//1.02), len(metadata))):
        added = metadata.values[idx]
        try:
            image = io.imread(added[1]) 
        except:  
            src = rio.open(added[1])
            image = src.read()

        for ch in range(3):
            added[5+ch] = np.mean(image[:,:,ch])
            added[8+ch] = np.std(image[:,:,ch])
            added[11+ch] = np.max(image[:,:,ch])
            added[14+ch] = np.min(image[:,:,ch])
        metadata.loc[idx] = added
    # metadata.to_csv(savepath, index=False)
    return metadata
make_stat(metadata, savepath='metadata3.csv')

 81%|████████  | 545/671 [00:04<00:01, 114.16it/s]/beegfs/desy/user/hailudaw/anacon/envs/tor/lib/python3.8/site-packages/rasterio/__init__.py:277: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  dataset = DatasetReader(path, driver=driver, sharing=sharing, **kwargs)
100%|██████████| 671/671 [00:05<00:00, 113.26it/s]


,Image,file,label,dataset,set,mean1,mean2,mean3,std1,std2,std3,max1,max2,max3,min1,min2,min3
0,BA_47.jpg,/beegfs/desy/user/hailudaw/blamad/Datasets/Ace...,basophil,Ace_20,train,0,0,0,0,0,0,0,0,0,0,0,0
1,BA_580.jpg,/beegfs/desy/user/hailudaw/blamad/Datasets/Ace...,basophil,Ace_20,train,0,0,0,0,0,0,0,0,0,0,0,0
2,BA_1223.jpg,/beegfs/desy/user/hailudaw/blamad/Datasets/Ace...,basophil,Ace_20,train,218.877005,183.383402,182.156451,43.775379,51.811281,20.213683,255,249,220,42,0,97
3,BA_1581.jpg,/beegfs/desy/user/hailudaw/blamad/Datasets/Ace...,basophil,Ace_20,train,216.585376,184.366713,180.041598,46.090832,54.040395,21.959705,255,251,220,41,0,93
4,BA_2035.jpg,/beegfs/desy/user/hailudaw/blamad/Datasets/Ace...,basophil,Ace_20,train,226.314264,195.118886,187.064838,48.624035,56.543244,21.614798,255,243,215,43,0,95
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34198,1646.TIF,/beegfs/desy/user/hailudaw/blamad/Datasets/WBC...,DATA-VAL,WBC1,train,171.271918,141.463903,186.089639,45.780371,71.56928,42.367915,253,239,254,78,26,93
34199,1647.TIF,/beegfs/desy/user/hailudaw/blamad/Datasets/WBC...,DATA-VAL,WBC1,train,189.248276,174.401391,200.254099,48.557436,60.127996,40.544286,255,255,255,47,20,80
34200,1648.TIF,/beegfs/desy/user/hailudaw/blamad/Datasets/WBC...,DATA-VAL,WBC1,train,223.107482,210.804446,223.476635,58.023042,70.529126,54.179214,255,255,255,23,15,32
34201,1649.TIF,/beegfs/desy/user/hailudaw/blamad/Datasets/WBC...,DATA-VAL,WBC1,train,181.579825,162.621588,192.673864,50.808985,63.212491,40.897169,255,255,255,54,23,97


In [82]:
metadata.values[:,].tolist()[34084]

['1527.TIF',
 '/beegfs/desy/user/hailudaw/blamad/Datasets/WBC1/DATA-VAL/1527.TIF',
 'DATA-VAL',
 'WBC1',
 'train',
 223.68995255586165,
 194.16443220079583,
 184.60173706764616,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

###  Authors

> Armin Gruber

> Ali Boushehri

> Christina Bukas

> Dawit Hailu